#### TOBIG'S 14기 정규세션 4주차 SVM 
### ASSIGNMENT1. Multiclass SVM 구현

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [58]:
from sklearn.model_selection import train_test_split #test/train 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [59]:
scaler = StandardScaler() #scaling
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [60]:
y_train = pd.get_dummies(y_train) #one hot encoding
y_test_one = pd.get_dummies(y_test)

In [61]:
y_train

,setosa,versicolor,virginica
110,0,0,1
69,0,1,0
148,0,0,1
39,1,0,0
53,0,1,0
...,...,...,...
64,0,1,0
91,0,1,0
81,0,1,0
51,0,1,0


In [62]:
# One VS Rest 구현을 위해 1) class가 0 or not 2)class가 1 or not을 구분하기 위한 머신 등을 미리 만들어주자
svm_1 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_2 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_3 = SVC(kernel ='rbf', C = 5, gamma = 5)

In [63]:
svm_1.fit(X_train,y_train.iloc[:,0]) # 데이터 클레스가 0 or not 구분해주는 머신
svm_2.fit(X_train,y_train.iloc[:,1]) # 데이터 클레스가 1 or not 구분해주는 머신
svm_3.fit(X_train,y_train.iloc[:,2]) # 데이터 클레스가 2 or not 구분해주는 머신
print(svm_1.predict(X_test)) #데이터 클래스가 0 or not을 구분해주는 애를 통해서 테스트 데이터의 클래스가 0인지, 0이 아닌인지 예측해보자

print(svm_1.decision_function(X_test)) #decision_function hyperplane과의 거리를 구하는 방법(필요하다면 활용해주세요!)

[0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[-1.12359969 -0.86782512 -0.65599247 -0.50194294 -0.76541147 -0.8819188
  1.07735938 -0.99156769  0.50201986 -0.9984315  -0.84532712  0.17062549
  0.34917127 -0.9813287  -0.72783399 -0.93313988  1.28153212 -0.56827872
 -0.73092732 -0.99670034  0.43553308 -0.96967771 -0.83939495 -1.03305682
 -0.75566609  1.13888006  0.42965012 -1.04268452 -0.93608147 -1.06090982]


In [64]:
# 부호가 모든 같은 경우가 있는가? > 모두 동점인 경우가 생길 것
for i in range(len(X_test)):
    # ~. decision_function을 이용하면 해당 데이터가 하이퍼플레인으로부터 얼마나 떨어져있는지 '거리'가 나온다!
    # 다음은 그 값의 부호를 이용해 모두가 동점인 경우가 있는지 출력하는 함수 
    if (np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i])) and (np.sign(svm_2.decision_function(X_test)[i]) == np.sign(svm_3.decision_function(X_test)[i])):
        print(i)

3
17
18


$ \sqrt{margin} $ = 값은 -1 또는 1 이다.
모든 class의 $ \sqrt{margin} $ 값이 같다면, 이는 모두가 동점인 경우를 의미한다.

* 동점이 아닌 경우

In [82]:
print(np.sign(svm_1.decision_function(X_test)[1]))
print(np.sign(svm_2.decision_function(X_test)[1]))
print(np.sign(svm_3.decision_function(X_test)[1]))

-1.0
1.0
-1.0


* 동점인 경우

In [83]:
print(np.sign(svm_1.decision_function(X_test)[3]))
print(np.sign(svm_2.decision_function(X_test)[3]))
print(np.sign(svm_3.decision_function(X_test)[3]))

-1.0
-1.0
-1.0


### OvR SVM

One vs Rest SVM 은 제일 많이 승리한 것에 대해 **투표**로 결정하는 method이다.  
decision_function을 통해 구한 margin값이 제일 큰 class를 선택한다. 
이 점을 참고하여 다중 SVM을 구현한 결과는 다음과 같다.  
(Class 개수 = SVC 모델 개수)

In [78]:
## Case 1 : One vs Rest SVM을 이부분에 구현해주세요 위 결과들을 이용해서 multi class SVM을 직접 구현해주세요! 하드코딩이 하시기 편할겁니다.

result = ['0' for i in range(X_test.shape[0])]
result1 = svm_1.decision_function(X_test) ; result2 = svm_2.decision_function(X_test) ; result3 = svm_3.decision_function(X_test)

for i in range(X_test.shape[0]):
    if max(result1[i], result2[i], result3[i]) == result1[i]:
        result[i] = 'setosa'
    elif max(result1[i], result2[i], result3[i]) == result2[i]:
        result[i] = 'versicolor'
    elif max(result1[i], result2[i], result3[i]) == result3[i]:
        result[i] = 'virginica'
        
print(result)
accuracy_score(y_test, result)

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


0.8666666666666667

### OvO SVM  

모든 Class에 대해 1:1 로 binary 분류를 하고, 제일 많이 승리한 것에 대해 투표로 결정하는 모델.  
Class 개수 = N일 때, N(N-1)/2 개의 모델이 필요하다.

In [88]:
# Case 2 : One vs One SVM을 이 부분에 구현해주세요. (선택사항)

svm_5 = SVC(decision_function_shape='ovo', kernel='rbf', C = 5, gamma = 5)

svm_5.fit(X_train,y_train)

ValueError: bad input shape (120, 3)

In [69]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요
from sklearn.model_selection import train_test_split #데이터셋 분리
from sklearn.metrics import accuracy_score
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

accuracy_score(y_test_2,y_pred)

0.8666666666666667